In [0]:
DECLARE OR REPLACE VARIABLE schema_use STRING DEFAULT REPLACE(SPLIT(current_user(), '@')[0], '.', '_');

In [0]:
USE IDENTIFIER("fhir_workshop." || schema_use);

In [0]:
SELECT current_catalog(), current_schema();

In [0]:
SHOW VOLUMES;

In [0]:
DECLARE OR REPLACE VARIABLE list_stmnt STRING;

SET VARIABLE list_stmnt = "LIST '/Volumes/fhir_workshop/" || schema_use || "/landing/';";

SELECT list_stmnt;

In [0]:
EXECUTE IMMEDIATE list_stmnt;

In [0]:
DROP TABLE IF EXISTS fhir_bronze;

In [0]:
DECLARE OR REPLACE VARIABLE create_streaming_bronze_stmnt STRING;

SET VARIABLE create_streaming_bronze_stmnt = "
CREATE OR REFRESH STREAMING TABLE fhir_bronze 
COMMENT 'Ingest FHIR JSON records as Full Text STRING'
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = 'true',
  'delta.enableDeletionVectors' = 'true',
  'delta.enableRowTracking' = 'true',
  'quality' = 'bronze'
)
AS SELECT
  _metadata as file_metadata
  ,* 
FROM STREAM read_files(
  '/Volumes/fhir_workshop/" || schema_use || "/landing/'
  ,format => 'text'
  ,wholeText => true
)
";

SELECT create_streaming_bronze_stmnt;

In [0]:
EXECUTE IMMEDIATE create_streaming_bronze_stmnt;